# Asistente Legal (LangChain + LLM)

Este notebook implementa el proyecto: "Asistente legal / normativo" para consultar leyes colombianas.

Objetivo de esta sección: preparar el entorno e importar las librerías necesarias. Sigue las celdas en orden.


In [30]:
##!uv add "langchain[openai]" dotenv ipywidgets chromadb faiss-cpu PyPDF2

## installar dependencias con pip
#%pip install -q "langchain[openai]" python-dotenv ipywidgets chromadb faiss-cpu PyPDF2
#!pip install groq langchain-groq


In [36]:
# Imports y configuración inicial
#from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_groq import ChatGroq
from os import getenv
from dotenv import load_dotenv

# Cargar variables de entorno desde .env
load_dotenv()

# Prompt template de ejemplo
template = """Question: {question}
Answer: Let's think step by step."""

prompt = PromptTemplate(template=template, input_variables=["question"])

# Inicialización del LLM (no ejecutar si no tiene las variables de entorno configuradas)
#llm = ChatOpenAI(
 #   api_key=getenv("OPENAPI_API_KEY"),
  #  base_url=getenv("OPENAPI_BASE_URL"),
   # model="google/gemini-2.5-flash-lite-preview-09-2025",
#)
llm = ChatGroq(
    api_key=os.getenv("GROQ_API_KEY"),
    model="llama-3.1-8b-instant"  # Groq soporta modelos como llama3-8b-8192 o mixtral-8x7b
)
# Ejemplo de uso (descomente si tiene las claves configuradas)
#question = "¿Qué equipo de la NFL ganó el Super Bowl en el año en que nació Justin Bieber?"
#print(llm.invoke(input=question, config={"prompt": prompt}))


content='Justin Bieber nació el 1 de marzo de 1994. El Super Bowl XXVIII se jugó en 1994 y fue ganado por el equipo de la NFL Dallas Cowboys.' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 42, 'prompt_tokens': 57, 'total_tokens': 99, 'completion_time': 0.065111534, 'prompt_time': 0.004080973, 'queue_time': 0.192308378, 'total_time': 0.069192507}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_50a6be1b6f', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None} id='run--66095e30-4306-4c5d-a5fe-c8a13ad2b4a2-0' usage_metadata={'input_tokens': 57, 'output_tokens': 42, 'total_tokens': 99}


In [6]:
# Loaders: widget FileUpload y helpers para parsear archivos subidos y leer `data/`
import ipywidgets as widgets
from IPython.display import display
import os
from io import BytesIO

# Widget para subir archivos
uploader = widgets.FileUpload(accept='.txt,.md,.pdf', multiple=True)
display(widgets.VBox([widgets.Label("Sube archivos legales (.pdf, .txt, .md)"), uploader]))

# Output para previsualización
output = widgets.Output()
display(output)


def parse_uploaded_files(uploader_widget):
    """Devuelve una lista de dicts {'filename', 'text'} extraídos de los archivos subidos."""
    docs = []
    # La estructura de uploader.value depende del frontend; manejar ambos casos
    items = getattr(uploader_widget, 'value', {}) or {}
    # En algunos entornos items es lista, en otros dict
    try:
        iterator = items.items()
    except Exception:
        # intentamos tratar como lista
        iterator = [(f.get('name', f"file_{i}"), f) for i, f in enumerate(items)]

    for name, fileinfo in iterator:
        # fileinfo puede ser dict con 'content' o directamente bytes
        content = fileinfo.get('content') if isinstance(fileinfo, dict) else fileinfo
        text = ''
        if name.lower().endswith('.pdf'):
            try:
                from PyPDF2 import PdfReader
                reader = PdfReader(BytesIO(content))
                pages = [p.extract_text() or '' for p in reader.pages]
                text = '\n'.join(pages)
            except Exception as e:
                text = f"[ERROR] No se pudo leer PDF ({name}): {e}"
        else:
            try:
                if isinstance(content, bytes):
                    text = content.decode('utf-8')
                else:
                    text = str(content)
            except Exception as e:
                text = f"[ERROR] decodificando {name}: {e}"
        docs.append({'filename': name, 'text': text})
    return docs


def on_preview_clicked(b):
    with output:
        output.clear_output()
        docs = parse_uploaded_files(uploader)
        if not docs:
            print('No se han encontrado archivos subidos.')
            return
        for d in docs:
            print(f"--- {d['filename']} ---")
            print(d['text'][:1500])
            print('\n')

preview_btn = widgets.Button(description='Previsualizar archivos')
preview_btn.on_click(on_preview_clicked)
display(preview_btn)


def load_local_docs(data_dir='data'):
    """Lee archivos de `data/` y devuelve lista de dicts {'filename','text'}"""
    docs = []
    if not os.path.exists(data_dir):
        print(f"No existe la carpeta '{data_dir}'. Crea la carpeta y añade documentos de ejemplo.")
        return docs
    for fn in sorted(os.listdir(data_dir)):
        path = os.path.join(data_dir, fn)
        if not os.path.isfile(path):
            continue
        text = ''
        if fn.lower().endswith('.pdf'):
            try:
                from PyPDF2 import PdfReader
                reader = PdfReader(path)
                pages = [p.extract_text() or '' for p in reader.pages]
                text = '\n'.join(pages)
            except Exception as e:
                text = f"[ERROR] leyendo PDF {fn}: {e}"
        else:
            try:
                with open(path, 'r', encoding='utf-8') as f:
                    text = f.read()
            except Exception as e:
                text = f"[ERROR] leyendo {fn}: {e}"
        docs.append({'filename': fn, 'text': text})
    return docs

# Ejemplo de uso: docs = load_local_docs('data')
# print(docs[0]['text'][:500]) if docs else print('No hay documentos en data/')


Output()

Button(description='Previsualizar archivos', style=ButtonStyle())

## Sección: Loaders (carga de documentos)

En esta sección implementaremos los loaders que permiten: (1) subir archivos locales mediante un widget, y (2) cargar archivos desde la carpeta `data/` del repositorio. Más adelante convertiremos estos textos en chunks y los indexaremos en una vector DB.

Instrucciones rápidas:
- Usa el widget para subir `.pdf`, `.txt` o `.md`.
- Pulsa "Previsualizar archivos" para ver el texto extraído (o errores si falta una dependencia de PDF).

In [7]:
# Preprocesamiento: splitters y creación de chunks
# Carga documentos desde `data/`, aplica un splitter y guarda los chunks en `DOCUMENT_CHUNKS`.
from collections import Counter

# Intentar importar el splitter de LangChain; si la API cambia, manejar el fallback
try:
    from langchain.text_splitter import RecursiveCharacterTextSplitter
except Exception:
    try:
        # versión alternativa
        from langchain.text_splitter import CharacterTextSplitter as RecursiveCharacterTextSplitter
    except Exception:
        RecursiveCharacterTextSplitter = None


def create_chunks(docs, chunk_size=1000, chunk_overlap=200):
    """Devuelve lista de dicts {'source','chunk_index','text'}"""
    if RecursiveCharacterTextSplitter is None:
        raise ImportError("No se encontró RecursiveCharacterTextSplitter. Instala langchain actualizado.")
    splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    all_chunks = []
    for d in docs:
        text = d.get('text') or ''
        if not text.strip():
            # texto vacío o error al leer
            continue
        texts = splitter.split_text(text)
        for i, t in enumerate(texts):
            all_chunks.append({'source': d.get('filename', 'unknown'), 'chunk_index': i, 'text': t})
    return all_chunks

# Cargar documentos locales desde data/
DOCUMENTS = load_local_docs('data')
print(f"Documentos cargados: {len(DOCUMENTS)}")

# Crear chunks
DOCUMENT_CHUNKS = create_chunks(DOCUMENTS, chunk_size=1000, chunk_overlap=200) if DOCUMENTS else []
print(f"Total chunks generados: {len(DOCUMENT_CHUNKS)}")

# Mostrar conteo por documento y un ejemplo
if DOCUMENT_CHUNKS:
    per_doc = Counter([c['source'] for c in DOCUMENT_CHUNKS])
    for doc, cnt in per_doc.items():
        print(f" - {doc}: {cnt} chunks")
    print('\nEjemplo (primer chunk, 500 chars):')
    print(DOCUMENT_CHUNKS[0]['text'][:500])
else:
    print('No hay chunks (documentos vacíos o problema al leer los archivos).')

# Ahora `DOCUMENTS` y `DOCUMENT_CHUNKS` están listos para embeddings / indexado.


Object ID 7606,0 ref repaired
Object ID 1,0 ref repaired
Object ID 2,0 ref repaired
Object ID 7582,0 ref repaired
Object ID 7584,0 ref repaired
Object ID 7583,0 ref repaired
Object ID 7589,0 ref repaired
Object ID 7591,0 ref repaired
Object ID 7590,0 ref repaired
Object ID 7596,0 ref repaired
Object ID 7598,0 ref repaired
Object ID 7597,0 ref repaired
Object ID 4,0 ref repaired
Object ID 7604,0 ref repaired
Object ID 6,0 ref repaired
Object ID 8,0 ref repaired
Object ID 10,0 ref repaired
Object ID 12,0 ref repaired
Object ID 14,0 ref repaired
Object ID 16,0 ref repaired
Object ID 18,0 ref repaired
Object ID 20,0 ref repaired
Object ID 22,0 ref repaired
Object ID 24,0 ref repaired
Object ID 26,0 ref repaired
Object ID 28,0 ref repaired
Object ID 30,0 ref repaired
Object ID 32,0 ref repaired
Object ID 34,0 ref repaired
Object ID 36,0 ref repaired
Object ID 38,0 ref repaired
Object ID 40,0 ref repaired
Object ID 42,0 ref repaired
Object ID 44,0 ref repaired
Object ID 46,0 ref repaired
Obj

Documentos cargados: 2
Total chunks generados: 1215
 - Constitución_Política_1_de_1991_Asamblea_Nacional_Constituyente.pdf: 1050 chunks
 - reglamento_academi_pregrado.pdf: 165 chunks

Ejemplo (primer chunk, 500 chars):
Departamento Administrativo de la Función Pública
Constitución Política 1 de 1991 Asamblea
Nacional Constituyente1 EVA - Gestor Normativo
Constitución Política 1 de 1991 Asamblea Nacional
Constituyente
Los datos publicados tienen propósitos exclusivamente informativos. El Departamento Administrativo de la Función Pública no se hace
responsable de la vigencia de la presente norma. Nos encontramos en un proceso permanente de actualización de los contenidos.
CONSTITUCIÓN POLITICA DE LA REPUBLICA DE


In [15]:
%pip install -q --upgrade langchain-openai


Note: you may need to restart the kernel to use updated packages.


In [16]:
# Embeddings + VectorStore (Chroma preferido, FAISS fallback) y pipeline RAG
import textwrap
# Requerir DOCUMENT_CHUNKS
try:
    len(DOCUMENT_CHUNKS)
except NameError:
    raise NameError('DOCUMENT_CHUNKS no está definido. Ejecuta la celda de splitters antes de esta.')

# Import embeddings
openAIEmbeddings = None
try:
    from langchain.embeddings import OpenAIEmbeddings
    openAIEmbeddings = OpenAIEmbeddings()
except Exception:
    try:
        from langchain.embeddings.openai import OpenAIEmbeddings
        openAIEmbeddings = OpenAIEmbeddings()
    except Exception:
        openAIEmbeddings = None

# Import vectorstores
Chroma = None
FAISS = None
try:
    from langchain.vectorstores import Chroma
    Chroma = Chroma
except Exception:
    try:
        from langchain.vectorstores import FAISS
        FAISS = FAISS
    except Exception:
        Chroma = None
        FAISS = None

if openAIEmbeddings is None:
    raise ImportError('No se encontró OpenAIEmbeddings. Instala/actualiza langchain o ajusta el código según la versión.')

# Preparar textos y metadatos
texts = [c['text'] for c in DOCUMENT_CHUNKS]
metadatas = [{'source': c['source'], 'chunk_index': c['chunk_index']} for c in DOCUMENT_CHUNKS]

# Inicializar embeddings
emb = openAIEmbeddings()

# Construir o cargar vectorstore
vectordb = None
try:
    if Chroma is not None:
        # persist_directory crea/usa la carpeta chroma_db
        vectordb = Chroma.from_texts(texts, embedding=emb, metadatas=metadatas, collection_name='laws', persist_directory='chroma_db')
        try:
            vectordb.persist()
        except Exception:
            pass
    else:
        # FAISS fallback (in-memory)
        from langchain.vectorstores import FAISS
        vectordb = FAISS.from_texts(texts, embedding=emb, metadatas=metadatas)
except Exception as e:
    raise RuntimeError(f'Error creando vectorstore: {e}')

print('Vectorstore preparado.')

# Helper: recuperar documentos (devuelve lista de objetos con page_content y metadata)
def retrieve(query, k=4):
    # similarity_search devuelve objetos Document o similares
    try:
        results = vectordb.similarity_search(query, k=k)
    except Exception:
        # Algunas versiones usan similarity_search_with_relevance_scores
        try:
            results = [r[0] for r in vectordb.similarity_search_with_relevance_scores(query, k=k)]
        except Exception as e:
            raise RuntimeError(f'No se pudo ejecutar la búsqueda: {e}')
    return results

# Pipeline RAG: sintetizar respuesta con LLM usando contexto recuperado

def rag_answer(query, k=4):
    docs = retrieve(query, k=k)
    if not docs:
        return {'answer': 'No se encontraron documentos relevantes.', 'citations': []}

    # Construir contexto concatenando fragmentos recuperados con metadata
    ctx_parts = []
    citations = []
    for d in docs:
        # compatibilidad: Document tiene page_content y metadata
        text = getattr(d, 'page_content', None) or d.get('text') if isinstance(d, dict) else str(d)
        meta = getattr(d, 'metadata', None) or (d if isinstance(d, dict) else {})
        src = meta.get('source', meta.get('file_name', 'unknown'))
        idx = meta.get('chunk_index')
        citations.append({'source': src, 'chunk_index': idx})
        snippet = textwrap.shorten(text, width=1000, placeholder='...')
        ctx_parts.append(f"Source: {src} (chunk {idx})\n{snippet}")

    context = '\n\n'.join(ctx_parts)

    # Prompt final
    final_prompt = f"Context:\n{context}\n\nQuestion: {query}\n\nAnswer conciso en español. Indica al final las citas como [source:filename chunk_index]."

    # Llamada al LLM (usar llm.invoke si está disponible)
    try:
        resp = llm.invoke(input=final_prompt)
        # si resp es un objeto complejo, intentar extraer texto
        answer_text = str(resp)
    except Exception:
        try:
            # fallback: si llm tiene método __call__
            answer_text = llm(final_prompt)
        except Exception as e:
            answer_text = f"[ERROR llamando al LLM: {e}]"

    return {'answer': answer_text, 'citations': citations}

# Widget simple para consultas RAG
query_box = widgets.Text(placeholder='Escribe tu pregunta sobre la ley (ej: ¿Qué artículo protege la libertad de expresión?)', description='Pregunta:', layout=widgets.Layout(width='80%'))
ask_btn = widgets.Button(description='Consultar')
out = widgets.Output()


def on_ask(b):
    with out:
        out.clear_output()
        q = query_box.value.strip()
        if not q:
            print('Escribe una pregunta primero.')
            return
        print('Buscando documentos relevantes...')
        res = rag_answer(q, k=4)
        print('\n--- RESPUESTA ---\n')
        print(res['answer'])
        print('\n--- CITAS ---')
        for c in res['citations']:
            print(f"- {c['source']} (chunk {c['chunk_index']})")

ask_btn.on_click(on_ask)

display(widgets.VBox([query_box, ask_btn, out]))

print('Interfaz RAG lista. Escribe una pregunta y pulsa "Consultar".')


TypeError: 'OpenAIEmbeddings' object is not callable

In [29]:
# CELDA: inicializar ChatOpenAI (robusta ante distintos estilos de llamada)
import os
from langchain_openai import ChatOpenAI

MODEL = os.getenv("OPENAI_MODEL", "gpt-3.5-turbo")  # cambia aquí si tienes otro modelo
API_KEY = os.getenv("OPENAI_API_KEY")

print("Usando modelo:", MODEL)

if not API_KEY:
    raise RuntimeError("OPENAI_API_KEY no encontrada. Define la variable de entorno antes de continuar.")

# Intenta inicializar el cliente LLM
try:
    llm = ChatOpenAI(api_key=API_KEY, model=MODEL)
    print("LLM inicializado (ChatOpenAI). Probando llamada breve...")
    # prueba de llamada con distintos estilos de interfaz (invoke / __call__)
    prueba = None
    try:
        prueba = llm.invoke("Saluda brevemente en español.")
        respuesta = getattr(prueba, "content", None) or str(prueba)
    except Exception:
        # fallback a llamada directa
        try:
            prueba = llm("Saluda brevemente en español.")
            respuesta = getattr(prueba, "content", None) or str(prueba)
        except Exception as err:
            raise RuntimeError(f"Error llamando al LLM tras inicializar: {err}")
    print("Respuesta de prueba:", respuesta)
except Exception as e:
    # captura y muestrame el error completo (útil para depuración)
    raise RuntimeError(f"Error inicializando ChatOpenAI (revisa modelo/API key): {e}")


Usando modelo: gpt-3.5-turbo
LLM inicializado (ChatOpenAI). Probando llamada breve...


C:\Users\HP\AppData\Local\Temp\ipykernel_18300\761360218.py:25: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  prueba = llm("Saluda brevemente en español.")


RuntimeError: Error inicializando ChatOpenAI (revisa modelo/API key): Error llamando al LLM tras inicializar: 'str' object has no attribute 'content'

In [ ]:
##WILSON ROJAS HIZÓ DESDE ACÁ. LA API KEY DE CHAT OPENAI NO ME SIRVE ENTONCES USÉ LA DE GROQ. ARRIBA ESTA EL LLAMADO , SI VAN A
##USAR LA DE CHATOPENAI , SOLO COMENTÉN MIS LINEAS DE GROQ , NO LAS ELIMINEN. 

In [43]:
##SECCION 5 PARSES 
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.prompts import PromptTemplate

# Definir los campos esperados en la salida
response_schemas = [
    ResponseSchema(name="answer", description="Respuesta concisa a la consulta legal en español."),
    ResponseSchema(name="citations", description="Lista de citas con formato [doc:filename, chunk:index]."),
    ResponseSchema(name="follow_up", description="true/false indicando si el usuario debería hacer una pregunta de seguimiento."),
]

# Crear el parser estructurado
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

# Instrucciones para el modelo
format_instructions = output_parser.get_format_instructions()

# Prompt template que exige formato
parser_prompt = PromptTemplate(
    template="""
Eres un asistente legal colombiano. Responde siempre en formato JSON.

Pregunta: {question}

{format_instructions}
""",
    input_variables=["question"],
    partial_variables={"format_instructions": format_instructions},
)

# Ejemplo de uso
question = "¿Qué artículo de la Constitución Colombiana protege la libertad de expresión?"

prompt_text = parser_prompt.format(question=question)
response = llm.invoke(prompt_text)

print("=== Respuesta cruda del LLM ===")
print(response.content)

try:
    parsed = output_parser.parse(response.content)
    print("\n=== Respuesta parseada (dict válido) ===")
    print(parsed)
except Exception as e:
    print("\n[ERROR] No se pudo parsear la salida:", e)


=== Respuesta cruda del LLM ===
```json
{
  "answer": "El artículo 20 de la Constitución Política de Colombia protege la libertad de expresión.",
  "citations": "[CNP:CONSTITUCION_DE_1991, art. 20]",
  "follow_up": "true"
}
```

Nota: El artículo 20 de la Constitución Política de Colombia establece que "Todas las personas tienen derecho a la libertad de expresión y de información, que incluye la libertad de buscar, recibir, recibir y difundir cualquier tipo de información y opiniones sin previas censura, sin sujeción a limitaciones de orden político, ideológico o religioso, ni a cualquier otra que no esté justificada por una ley que, entre otros requisitos, debe ser respetuosa de ese derecho y no afectar la fundamentales de la personalidad humana."

=== Respuesta parseada (dict válido) ===
{'answer': 'El artículo 20 de la Constitución Política de Colombia protege la libertad de expresión.', 'citations': '[CNP:CONSTITUCION_DE_1991, art. 20]', 'follow_up': 'true'}


In [44]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

# Definir un prompt que incluya historial de conversación
memory_prompt = PromptTemplate(
    input_variables=["history", "question"],
    template="""
Eres un asistente legal colombiano.
Responde en español de forma clara y breve.

Historial de conversación:
{history}

Nueva pregunta:
{question}

Respuesta:
"""
)

# Inicializar la memoria
memory = ConversationBufferMemory(memory_key="history")

# Construir el chain con memoria
conversation_chain = LLMChain(
    llm=llm,
    prompt=memory_prompt,
    memory=memory,
)

# Ejemplo de interacción multi-turno
print("Turno 1:")
resp1 = conversation_chain.run("Soy un estudiante de Ingenieria de sistemas y me llamo Wilson")
print(resp1)

print("\nTurno 2 (con memoria):")
resp2 = conversation_chain.run("¿Como me llamo y que estudio?")
print(resp2)



Turno 1:
¡Hola Wilson! Me alegra conocerte. ¿En qué puedo ayudarte hoy? ¿Tienes alguna inquietud o pregunta relacionada con tu carrera o un tema en particular?

Turno 2 (con memoria):
¡Hola Wilson! Me alegra que hayas recordado tus datos. Según nuestro historial de conversación, tu nombre es Wilson y estudias Ingeniería de Sistemas. ¿Quieres hablar sobre algo relacionado a tu carrera o necesitas orientación sobre algún tema en particular?


In [45]:
from langchain.agents import initialize_agent, Tool, AgentType
from langchain_core.prompts import PromptTemplate

# --- Angente React Basico ---

# Herramienta de búsqueda en vector DB (usa tu función retrieve/rag_answer)
def buscar_en_leyes(query: str) -> str:
    """Busca en la base de datos legal usando RAG."""
    res = rag_answer(query, k=3)
    return f"Respuesta (RAG): {res['answer']}"

tools = [
    Tool(
        name="BusquedaLegalRAG",
        func=buscar_en_leyes,
        description="Usa esta herramienta cuando la pregunta se relacione con leyes o documentos cargados."
    ),
    Tool(
        name="LLMGeneral",
        func=lambda q: llm.invoke(q).content,
        description="Usa esta herramienta cuando la pregunta NO sea legal o no requiera buscar documentos."
    )
]

# --- Inicializamos agente con tipo ReAct ---
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,   # muestra el razonamiento paso a paso
    max_iterations=3,
)

# --- Ejemplo de uso ---
print("Ejemplo 1: Pregunta legal (usa RAG)")
respuesta1 = agent.run("¿Qué artículo protege la libertad de expresión en Colombia?")
print(respuesta1)

print("\nEjemplo 2: Pregunta general (usa LLM)")
respuesta2 = agent.run("¿Quién fue Gabriel García Márquez?")
print(respuesta2)


Ejemplo 1: Pregunta legal (usa RAG)


> Entering new AgentExecutor chain...
Thought: La pregunta se refiere a una ley o documento específico en Colombia, por lo que es probable que deba buscar esta información en una base de datos legal.
Action: Buscar la información en la base de datos legal.
Action Input: query="libertad de expresión en Colombia"
Observation: Buscar la información en la base de datos legal. is not a valid tool, try one of [BusquedaLegalRAG, LLMGeneral].
Thought:Thought: Entiendo el error. La herramienta correcta para buscar información legal en una base de datos es BusquedaLegalRAG. 
Action: Utilizar la herramienta BusquedaLegalRAG.
Action Input: query="libertad de expresión en Colombia".
Observation: Utilizar la herramienta BusquedaLegalRAG. is not a valid tool, try one of [BusquedaLegalRAG, LLMGeneral].
Thought:Parece que cometí un error en la notación. Aquí está la respuesta corregida:

Question: ¿Qué artículo protege la libertad de expresión en Colombia?
Thought:

In [40]:
# ===== CELDA: Definir retrieve + rag_answer + re-crear agente ReAct =====
import textwrap
import traceback

# 1) Comprobaciones básicas
if 'llm' not in globals():
    raise NameError("No existe la variable 'llm'. Inicializa tu LLM (ChatGroq/ChatOpenAI) antes de ejecutar esta celda.")

has_vectordb = ('vectordb' in globals() and globals()['vectordb'] is not None)
has_chunks = ('DOCUMENT_CHUNKS' in globals() and bool(DOCUMENT_CHUNKS))

# 2) Funciones de recuperación (vectordb o fallback por keywords)
def retrieve_by_keywords(query: str, k: int = 4):
    """Fallback simple: cuenta ocurrencias en DOCUMENT_CHUNKS."""
    if not has_chunks:
        return []
    q = query.lower()
    scored = []
    for c in DOCUMENT_CHUNKS:
        text = c.get('text','') or ''
        score = text.lower().count(q)  # simple heurística
        scored.append({'text': text, 'score': score, 'metadata': {'source': c.get('source'), 'chunk_index': c.get('chunk_index')}})
    scored = sorted(scored, key=lambda x: x['score'], reverse=True)
    return scored[:k]

def retrieve_from_vectordb(query: str, k: int = 4):
    """Intentar usar vectordb (Chroma/FAISS) si está disponible."""
    try:
        # preferimos similarity_search_with_relevance_scores si existe
        if hasattr(vectordb, "similarity_search_with_relevance_scores"):
            results = vectordb.similarity_search_with_relevance_scores(query, k=k)
            out = []
            for doc, score in results:
                text = getattr(doc, "page_content", None) or (doc.get('text') if isinstance(doc, dict) else str(doc))
                meta = getattr(doc, "metadata", None) or (doc if isinstance(doc, dict) else {})
                out.append({'text': text, 'score': float(score), 'metadata': meta})
            return out
        else:
            results = vectordb.similarity_search(query, k=k)
            out = []
            for r in results:
                text = getattr(r, "page_content", None) or (r.get('text') if isinstance(r, dict) else str(r))
                meta = getattr(r, "metadata", None) or (r if isinstance(r, dict) else {})
                out.append({'text': text, 'score': None, 'metadata': meta})
            return out
    except Exception as e:
        print("⚠️ Error buscando en vectordb:", e)
        print("Se usará fallback por keywords (DOCUMENT_CHUNKS).")
        return retrieve_by_keywords(query, k=k)

def retrieve(query: str, k: int = 4):
    return retrieve_from_vectordb(query, k) if has_vectordb else retrieve_by_keywords(query, k)

# 3) Wrapper LLM compatible con .invoke() y llamada directa
def llm_call(prompt: str, **kwargs) -> str:
    """Llama al LLM y devuelve texto; maneja distintos formatos de respuesta."""
    try:
        resp = llm.invoke(prompt, **kwargs)
        return getattr(resp, "content", None) or str(resp)
    except Exception:
        try:
            resp = llm(prompt, **kwargs)
            return getattr(resp, "content", None) or str(resp)
        except Exception as e:
            return f"[ERROR en LLM]: {e}"

# 4) rag_answer robusto: recupera, arma contexto y llama al LLM
def rag_answer(query: str, k: int = 4):
    docs = retrieve(query, k)
    if not docs:
        return {'answer': 'No se encontraron documentos relevantes.', 'citations': []}

    ctx_parts = []
    citations = []
    for d in docs:
        text = d.get('text') if isinstance(d, dict) else str(d)
        meta = d.get('metadata', {}) if isinstance(d, dict) else {}
        src = meta.get('source', meta.get('filename', 'unknown'))
        idx = meta.get('chunk_index')
        score = d.get('score')
        citations.append({'source': src, 'chunk_index': idx, 'score': score})
        snippet = textwrap.shorten(text, width=1000, placeholder='...')
        ctx_parts.append(f"Source: {src} (chunk {idx}, score {score})\n{snippet}")

    context = "\n\n".join(ctx_parts)
    final_prompt = (
        "Eres un asistente legal. Usa SOLO el contexto provisto para responder la pregunta. "
        "Responde en español, de forma concisa, y al final incluye las citas en formato [source:filename chunk_index].\n\n"
        f"Contexto:\n{context}\n\n"
        f"Pregunta: {query}\n\n"
        "Respuesta:"
    )

    answer_text = llm_call(final_prompt)
    return {'answer': answer_text, 'citations': citations}

# 5) Reconstruir agente ReAct usando la herramienta que llama a rag_answer
from langchain.agents import initialize_agent, Tool, AgentType

def buscar_en_leyes_tool(q: str) -> str:
    try:
        r = rag_answer(q, k=3)
        return r.get('answer', '')
    except Exception as e:
        return f"[ERROR herramienta RAG]: {e}"

def llm_general_tool(q: str) -> str:
    try:
        return llm_call(q)
    except Exception as e:
        return f"[ERROR herramienta LLMGeneral]: {e}"

tools = [
    Tool(name="BusquedaLegalRAG", func=buscar_en_leyes_tool,
         description="Buscar en documentos legales indexados. Útil para consultas concretas sobre leyes y artículos."),
    Tool(name="LLMGeneral", func=llm_general_tool,
         description="Responder con el LLM sin usar la base de datos (general knowledge).")
]

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    max_iterations=3
)

print("✅ Definido rag_answer y recreado 'agent' con la herramienta RAG. Prueba con agent.run(...)")


✅ Definido rag_answer y recreado 'agent' con la herramienta RAG. Prueba con agent.run(...)


In [41]:
# Pruebas rápidas
print(agent.run("¿Qué artículo protege la libertad de expresión en Colombia?"))
print(agent.run("Resume brevemente el concepto de habeas corpus."))




> Entering new AgentExecutor chain...
Thought: Necesito encontrar un artículo legal que mencione la protección de la libertad de expresión en Colombia.
Action: BusquedaLegalRAG
Action Input: "libertad de expresión Colombia"
Observation: La Constitución Política de Colombia de 1991 establece que el Estado colombiano es democrático, participativo y pluralista, y que garantiza la dignidad humana, el trabajo y la solidaridad de las personas que lo integran (Artículo 1). Además, el artículo 20 de la Constitución establece que:

"La libertad de expresión consagrada en el artículo 19 de la Declaración Universal de los Derechos Humanos, comprende la libertad de buscar, recibir y difundir informaciones y opiniones de cualquier clase, así como la libertad de difundir ideas y opiniones sin sujeción a previa censura, sin embargo, el Estado garantiza el derecho a la información y a la protección de fuentes de información y periodistas, en el marco de sus atribuciones constitucionales y legales".
